<a href="https://colab.research.google.com/github/slz4025/twitter_latent_scams/blob/master/scorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a basic regex scorer that helps identify tweets that are more obviously scams.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
file_name = "filtered_test" #@param {type: "string"}

In [ ]:
processed_dir = "drive/My Drive/latent_scams/processed_data/"
analyzed_dir = "drive/My Drive/latent_scams/analyzed/"
data_file = "{}.json".format(file_name)
score_file = "scored_{}.tsv".format(file_name)

In [ ]:
import json
import pandas as pd
import re

In [ ]:
scam_words_list = [
    (re.compile(r"\bscam", re.IGNORECASE), 2),
    (re.compile(r"\bfraud", re.IGNORECASE), 2),
    (re.compile(r"\bthief\b", re.IGNORECASE), 0),
    (re.compile(r"\bthieves\b", re.IGNORECASE), 0),
    (re.compile(r"\btheft\b", re.IGNORECASE), 1),
    (re.compile(r"\bhacked\b", re.IGNORECASE), 3),
    (re.compile(r"\bhacker\b", re.IGNORECASE), 5),
    (re.compile(r"\bphish", re.IGNORECASE), 5),
    (re.compile(r"\bransomware\b", re.IGNORECASE), 5),
    (re.compile(r"\bmalware\b", re.IGNORECASE), 5),
    (re.compile(r"\bspam", re.IGNORECASE), 2),
    (re.compile(r"\bspoof", re.IGNORECASE), 3),
    (re.compile(r"\brobocall", re.IGNORECASE), 5),
    (re.compile(r"\bcounterfeit", re.IGNORECASE), 1),
    (re.compile(r"\bimpersonat", re.IGNORECASE), 3),
    (re.compile(r"\bcon(s\b|\b)", re.IGNORECASE), 1),
    (re.compile(r"\bphony\b", re.IGNORECASE), 0),
    (re.compile(r"\bphoney\b", re.IGNORECASE), 0),
    (re.compile(r"\bthreat", re.IGNORECASE), 1),
    (re.compile(r"\blure", re.IGNORECASE), 2),
    (re.compile(r"\bexploit", re.IGNORECASE), 2),
    (re.compile(r"\bprey", re.IGNORECASE), 3),
    (re.compile(r"\bmalicious\b", re.IGNORECASE), 4),
    (re.compile(r"\btarget", re.IGNORECASE), 1),
    (re.compile(r"\bvulnerable\b", re.IGNORECASE), 1),
    (re.compile(r"\bfall for\b", re.IGNORECASE), 0.5),
    (re.compile(r"\bmiracle", re.IGNORECASE), 1),
    (re.compile(r"\bclick", re.IGNORECASE), 0),
    (re.compile(r"\blink", re.IGNORECASE), 0),
    (re.compile(r"\bid\b", re.IGNORECASE), 0),
    (re.compile(r"\bidentity\b", re.IGNORECASE), 2.5),
    (re.compile(r"\bsham\b", re.IGNORECASE), 0),
    (re.compile(r"\blaunder", re.IGNORECASE), 1),
    (re.compile(r"\bdownload", re.IGNORECASE), 0.5),
    (re.compile(r"\bvoucher\b", re.IGNORECASE), 1),
    (re.compile(r"\brefund", re.IGNORECASE), 4),
    (re.compile(r"\bbrowser\b", re.IGNORECASE), 0),
    (re.compile(r"\binbox\b", re.IGNORECASE), 0.5),
    (re.compile(r"\bbreach", re.IGNORECASE), 3),
    (re.compile(r"\bcrime\b", re.IGNORECASE), 3),
    (re.compile(r"\bcriminal", re.IGNORECASE), 1.5),
    (re.compile(r"\bhack\b", re.IGNORECASE), 1.5),
    (re.compile(r"\bsteal\b", re.IGNORECASE), 1),
    (re.compile(r"\bstole\b", re.IGNORECASE), 1),
    (re.compile(r"\bsuspicious\b", re.IGNORECASE), 4),
    (re.compile(r"\bscheme", re.IGNORECASE), 0),
    (re.compile(r"\bvictim", re.IGNORECASE), 0),
    (re.compile(r"(\b|\bun)solicit", re.IGNORECASE), 1),
    (re.compile(r"\bpretend", re.IGNORECASE), 0),
    (re.compile(r"\bbitcoin\b", re.IGNORECASE), 0),
    (re.compile(r"\bcrypto", re.IGNORECASE), 2),
    (re.compile(r"\bdebit", re.IGNORECASE), 0),
    (re.compile(r"\bgiftcard", re.IGNORECASE), 0),
    (re.compile(r"\bemail", re.IGNORECASE), 1),
    (re.compile(r"\bfake", re.IGNORECASE), 0),
    (re.compile(r"\bfalse\b", re.IGNORECASE), 0),
    (re.compile(r"\btext", re.IGNORECASE), 1.5),
    (re.compile(r"\btxt", re.IGNORECASE), 0),
    (re.compile(r"message", re.IGNORECASE), 0),
    (re.compile(r"\bstimulus", re.IGNORECASE), 0),
    (re.compile(r"\btax", re.IGNORECASE), 0),
    (re.compile(r"\bcaller", re.IGNORECASE), 1),
    (re.compile(r"\bbuy", re.IGNORECASE), 0),
    (re.compile(r"\bcontact trace", re.IGNORECASE), 0),
    (re.compile(r"\bcomplaint\b", re.IGNORECASE), 0),
    (re.compile(r"\bfile", re.IGNORECASE), 0),
    (re.compile(r"\bwire", re.IGNORECASE), 0),
    (re.compile(r"\bweb\b", re.IGNORECASE), 0),
    (re.compile(r"\bwebpage", re.IGNORECASE), 0),
    (re.compile(r"\bloan", re.IGNORECASE), 0),
    (re.compile(r"\bsms\b", re.IGNORECASE), 1.5),
    (re.compile(r"(\b|\bun)authentic", re.IGNORECASE), 1.5),
    (re.compile(r"\blie(\b|s\b)", re.IGNORECASE), 0),
    (re.compile(r"\bliar", re.IGNORECASE), 0),
    (re.compile(r"\bad(\b|s\b)", re.IGNORECASE), 0),
    (re.compile(r"\badvertise", re.IGNORECASE), 0),
]
scam_words = {r.pattern:{"reg": r, "score": s} for r, s in scam_words_list}

In [ ]:
global score_df
score_df = pd.DataFrame()

In [ ]:
def score_text(body, hashtags):
    # for now, hashtags should be in body so just use body
    score = sum([v["score"] for k, v in scam_words.items() if v["reg"].search(body) is not None])
    rationale = [k for k, v in scam_words.items() if v["reg"].search(body) is not None]
    return score, rationale

def score(tweet):
    global score_df
    tweet_id = tweet["id_str"]
    score = 0
    rationale = []

    body = tweet["text"]
    hashtags = [h["text"] for h in tweet["entities"]["hashtags"]]
    if "extended_tweet" in tweet:
        body = tweet["extended_tweet"]["full_text"]
        hashtags = [h["text"] for h in tweet["extended_tweet"]["entities"]["hashtags"]]
    sub_score, sub_rationale = score_text(body, hashtags)

    score += sub_score
    rationale += sub_rationale 

    score_df = score_df.append({"tweet_id": tweet_id, "score": score, "rationale": rationale}, ignore_index = True)

In [ ]:
with open(processed_dir + data_file) as contents:
    count = 0
    for line in contents:
        line = line.strip()
        if line == '': continue
        t = json.loads(line)
        score(t)
        if count % 1000 == 0: print("Processed ", count)
        count += 1

Processed  0


In [ ]:
score_df.to_csv(
    sep="\t", path_or_buf=analyzed_dir + score_file
)